In [6]:
'''
Pull price data from Marketstack API
Created JR 09/10/2024
Requires API key from Marketstack. 
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 
           'FB', 'PG', 'AVGO', 'BRK-B', 
           'GOOG', 'LLY', 'TSLA', 'JPM', 
           'UNH', 'XOM', 'V']
# symbols = ['AAPL'] # Testing to confirm error in adj close price. 
end_date = "2024-09-30"  
start_date = "2019-09-30"  

# API EOD URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Function to fetch EOD data for a single symbol
def get_eod_data(symbol, start_date, end_date, api_key):
    
    all_data = []  
    limit = 1000  # confirmed max 1000 via marketstack docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbol,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit, # required - if this is missing only returns the first 1000. 
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data. 
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    df = pd.DataFrame(all_data)
    df['symbol'] = symbol  # Add a column for the symbol

    return df

# Initialize an empty DataFrame to store all the data
price_df = pd.DataFrame()

# Loop through each symbol and fetch its data
for symbol in symbols:
    symbol_df = get_eod_data(symbol, start_date, end_date, API_KEY)
    price_df = pd.concat([price_df, symbol_df], ignore_index=True)

print(price_df.head())

# Format the date
price_df = price_df[['date', 'adj_close', 'symbol']]
price_df['date'] = pd.to_datetime(price_df['date']).dt.date

# Switch to array with symbols 
price_df = price_df.pivot(index='date', columns='symbol', values='adj_close')

print(price_df.head())

# Save to csv to prevent having to make multiple calls whilst testing
price_df.to_csv('combined_eod_data.csv', index=True)

      open    high      low   close      volume  adj_high  adj_low  adj_close  \
0  230.155  233.00  229.655  233.00  52779736.0    233.00   229.65     233.00   
1  228.460  229.52  227.300  227.79  33993600.0    229.52   227.30     227.79   
2  227.300  228.50  225.410  227.52  36490267.0    228.50   225.41     227.52   
3  224.930  227.29  224.030  226.37  40779133.0    227.29   224.02     226.37   
4  228.645  229.35  225.740  227.37  41781114.0    229.35   225.73     227.37   

   adj_open  adj_volume  split_factor  dividend symbol exchange  \
0   230.040  54793391.0           1.0       0.0   AAPL     XNAS   
1   228.460  34025967.0           1.0       0.0   AAPL     XNAS   
2   227.300  36636707.0           1.0       0.0   AAPL     XNAS   
3   224.930  42308715.0           1.0       0.0   AAPL     XNAS   
4   228.645  43556068.0           1.0       0.0   AAPL     XNAS   

                       date  
0  2024-09-30T00:00:00+0000  
1  2024-09-27T00:00:00+0000  
2  2024-09-26T00:00:

In [1]:
'''
Pull price data from yahoo finance API
Created JR 16/10/2024
No key required, limits on api calls and professional usage. 
'''

import yfinance as yf
import pandas as pd

# List of tickers
symbols = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 
           'META', 'AVGO', 'BRK-B', 'PG',
           'GOOG', 'LLY', 'TSLA', 'JPM', 
           'UNH', 'XOM', 'V']

# Define the date range
start_date = '2019-09-30'
end_date = '2024-10-01'

# Download adjusted close price data
data = yf.download(symbols, start=start_date, end=end_date, 
                   progress=False)['Adj Close']

# Sort the columns alphabetically by ticker
data = data.reindex(sorted(data.columns), axis=1)
data = data.reset_index().rename(columns={'Date': 'date'})
data['date'] = pd.to_datetime(data['date']).dt.date

# Save the sorted DataFrame to a CSV file
data.to_csv('adjust_close.csv', index=False)

print(data.head())

Ticker        date       AAPL       AMZN       AVGO       BRK-B       GOOG  \
0       2019-09-30  54.180809  86.795502  23.788462  208.020004  60.800686   
1       2019-10-01  54.330784  86.782501  23.683336  206.710007  60.107388   
2       2019-10-02  52.968845  85.661499  23.271454  203.100006  58.687374   
3       2019-10-03  53.418785  86.221001  23.647152  204.979996  59.245998   
4       2019-10-04  54.916229  86.982498  24.250324  208.080002  60.301907   

Ticker         JPM         LLY        META        MSFT      NVDA          PG  \
0       101.043938  104.387283  177.545425  132.794464  4.330685  109.240562   
1        99.206619  103.575195  175.282242  130.922394  4.328944  108.775063   
2        97.231926   99.878731  174.075882  128.610901  4.305061  106.342232   
3        97.093452  100.149414  178.841537  130.167801  4.510809  106.921883   
4        99.196510  100.998871  179.908325  131.925293  4.527230  108.906792   

Ticker       TSLA         UNH           V        X

In [ ]:
'''
Pull price data from Tiingo. 
Replacement for Marketstack, which returns incorrect adjusted close prices. 
Requires API key from Tiingo
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Tiingo')

# Define the securities and date range
symbols = ['AAPL', 'NVDA', 'MSFT', 'AMZN', 
           'META', 'PG', 'AVGO', 'BRK-B', 
           'GOOG', 'LLY', 'TSLA', 'JPM', 
           'UNH', 'XOM', 'V']
# symbols = ['AAPL'] # Testing to confirm error in adj close price. 
end_date = "2024-09-30"  
start_date = "2019-09-30"  

# API EOD URL from Tiingo
url = 'https://api.tiingo.com/tiingo/daily/{symbol}/prices'

# Function to fetch EOD data for a single symbol
def get_eod_data(symbol, start_date, end_date, api_key):
    # Define parameters for the API request
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token {api_key}'
    }
    querystring = {
        'startDate': start_date,
        'endDate': end_date
    }

    # Make the request
    response = requests.get(url.format(symbol=symbol), headers=headers, params=querystring)

    # Check all data has been received
    if response.status_code == 200: # response received
        data = response.json()
        if not data:
            return pd.DataFrame()  # Return empty DataFrame if no data available

        # Convert the collected data to a pandas DataFrame
        df = pd.DataFrame(data)
        df['symbol'] = symbol  # Add a column for the symbol
        return df
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return pd.DataFrame()

# Initialize an empty DataFrame to store all the data
price_df = pd.DataFrame()

# Loop through each symbol and fetch its data
for symbol in symbols:
    symbol_df = get_eod_data(symbol, start_date, end_date, API_KEY)
    price_df = pd.concat([price_df, symbol_df], ignore_index=True)

# Check if the data is non-empty before proceeding
if not price_df.empty:
    # Format the date
    price_df = price_df[['date', 'adjClose', 'symbol']]
    price_df['date'] = pd.to_datetime(price_df['date']).dt.date

    # Switch to array with symbols 
    price_df = price_df.pivot(index='date', columns='symbol', values='adjClose')

    print(price_df.head())

    # Save to csv to prevent having to make multiple calls whilst testing
    price_df.to_csv('combined_eod_data_tiingo.csv', index=True)
else:
    print("No data available")

symbol           AAPL     AMZN       AVGO   BRK-B       GOOG         JPM  \
date                                                                       
2019-09-30  54.183676  86.7955  23.787121  208.02  60.799748  101.019278   
2019-10-01  54.333669  86.7825  23.682002  206.71  60.106461   99.182408   
2019-10-02  52.971638  85.6615  23.270141  203.10  58.686470   97.208202   
2019-10-03  53.421616  86.2210  23.645813  204.98  59.245090   97.070866   
2019-10-04  54.919124  86.9825  24.248957  208.08  60.300981   99.173390   

symbol             LLY        META        MSFT      NVDA          PG  \
date                                                                   
2019-09-30  104.394428  177.547244  132.787550  4.330605  109.215953   
2019-10-01  103.582274  175.284035  130.915554  4.328863  108.750569   
2019-10-02   99.885575  174.077655  128.604212  4.304980  106.318280   
2019-10-03  100.156293  178.843355  130.161025  4.510725  106.897814   
2019-10-04  101.005787  179.910154 